# MPAR - Rapport du rendu II
**01/04/24**

## Introduction

Ce notebook présente l'explication, la démonstration et les résultats des outils développés dans le cadre du cours de MPAR. Les résultats de la première partie du rendu, discutés en cours, ne seront pas abordés ici mais seront uniquement utilisés comme référence.

## Objectifs

1. Comprendre les concepts de **Model Checking** et de **SMC (Statistical Model Checking)** pour les **chaînes de Markov**.
2. Étendre ces concepts aux **MDP (Markov Decision Process)**.
3. Utiliser le **RL (Reinforcement Learning)** pour améliorer le model checking statistique.

## Contenu
1. **Changements dans le parser**
2. **Chapitre 2, Vérification Probabiliste**
3. **Chapitre 3, Modélisation Probabiliste et Apprentissage par Renforcement**

## 1. Changements dans le parser

Pour pouvoir utiliser des modèles .mdp avec des recompenses, il a fallut modifier notre parser. Pour ceci, les anciens modèles .mdp, sans récompenses, doivent continuer a fonctionnner sans altération.

De cette façon, nous avons inclu la ligne optionnelle `Rewards` avec des entrées dans le format `s:r` avec s un état et r une récompense entière. L'exemple `states_with_rewards` montre ceci.

Ceci a été fait en ajoutant l'entrée (defrewards) optionnele (?)  a la définition du programme, ayant `program: defstates defrewards? defactions transitions EOF;` et ayant la définition `defewrards : REWARDS ID ':' INT (',' ID ':' INT)* ';';`.

Après, nous avons ajouté l'attribut `rewards` à la classe `gramPrintListener`, défini a travers les fonctions `enterDefrewards` et `update_rewards`.

De cette façon, comme dans l'exemple vu en cours, il suffit d'importer la fonction `run` de l'archive `mdp.py` pour lire un fichier et creer un objet de la classe `gramPrintListener` qui contiendrat, en plus des attributs vus dans la prémière démostration, les attributs de récompenses.

In [6]:
from mdp import run as run_mdp # Fonction qui crée l'objet a être lu.

Exemple sans récompenses:

In [7]:
simu_mc = run_mdp(path = "prof_examples//simu-mc.mdp", return_printer=True, print_transactions=True) # Exemple sans récompenses

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['I', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']
Initialy declared actions: ['a']
Transition from I with no action and targets ['T1', 'T2'] with weights [1, 1]
Transition from T1 with no action and targets ['T3', 'T4'] with weights [1, 1]
Transition from T2 with no action and targets ['T5', 'T6'] with weights [1, 1]
Transition from T3 with no action and targets ['S1', 'T1'] with weights [1, 1]
Transition from T4 with no action and targets ['S2', 'S3'] with weights [1, 1]
Transition from T5 with no action and targets ['S4', 'S5'] with weights [1, 1]
Transition from T6 with no action and targets ['S6', 'T2'] with weights [1, 1]
Transition from S1 with no action and targets ['S1'] with weights [1]
Transition from S2 with no action and targets ['S2'] with weights [1]
Transition from S3 with no action and

Exemple avec récompenses:

In [8]:
states_with_rewards = run_mdp(path = "mdp_examples//states_with_rewards.mdp", return_printer=True, print_transactions=True) # Exemple avec récompenses

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['S0', 'S1', 'S2']
Initialy declared actions: ['a', 'b', 'c']
Transition from S0 with no action and targets ['S1', 'S2'] with weights [5, 5]
Transition from S1 with action b and targets ['S1', 'S0'] with weights [2, 8]
Transition from S1 with action a and targets ['S2', 'S0', 'S1', 'S3'] with weights [1, 3, 6, 2]
Transition from S2 with action c and targets ['S0', 'S1', 'S3'] with weights [5, 5, 10]
Transition from S2 with action d and targets ['S0', 'S3'] with weights [5, 7]
Transition from S3 with action e and targets ['S1', 'S2'] with weights [2, 2]

 ------- transactions df -------
  Origin Action   S0   S1   S2    S3
0     S0     NA  NaN    5    5   NaN
1     S1      b    8    2  NaN   NaN
2     S1      a    3    6    1   2.0
3     S2      c    5    5  NaN  10.0
4     S2      d    5  NaN  NaN   7.0
5     S3      e  NaN    2

Comme montré dans les exemples précedents, à chaque état nous avons un reward attribué. 

Si la récompense n'a pas été définie, notre parser attribut une valeur de zero. 

Il est un dictionnaire et peut être appelé facilement par l'attribut `rewards` comme suit:

In [9]:
print('states_with_reards.mdp', states_with_rewards.rewards)
print('simu_mc.mdp', simu_mc.rewards)

states_with_reards.mdp {'S0': 1, 'S1': 10, 'S2': 15, 'S3': 0}
simu_mc.mdp {'I': 0, 'T1': 0, 'T2': 0, 'T3': 0, 'T4': 0, 'T5': 0, 'T6': 0, 'S1': 0, 'S2': 0, 'S3': 0, 'S4': 0, 'S5': 0, 'S6': 0}


## 2. Vérification Probabiliste (Chapitre 2)

### Exercice 12: 
Identification des ensembles $S_0$, $S_1$ et $S_?$ pour une propriété $P(\diamond s)$ (fatalment $s$, où $s$ est un état).

# OBS: Está errado, mesmo corrigindo para MDPs, também dá erro se um estado passar por outro com prob 1. A ideia do algoritmo em si está errada pq só pega transições diretas, o que não faz sentido.

In [18]:
# Working for both MDPs and MCs.
def find_states(printer, target_state):
    """
    Identifies states leading to a target state with certainty (S_sure), states that may lead to the target state (S_may),
    and states that can never reach the target state (S_never) in a Markov chain represented by a DataFrame.

    Parameters:
    - printer - our object of class gramPrintListener
    - target_state (str): The state of interest to trace back from.

    Returns:
    - tuple of lists: A tuple containing three lists representing states that are sure to reach the target (S_sure),
                      states that may reach the target (S_may), and states that can never reach the target (S_never).
    """
    df = printer.transactions_prob
    
    # Initialize sets for S_sure, S_may, and S_never. We begin with all as never
    S_sure = set()          
    S_may = set()           
    S_never = set(printer.declared_states)   

    # The target state of in S_sure
    S_sure.add(target_state)
    S_never.remove(target_state)
    
    # States visited or to visit
    visited = set()
    to_visit = [target_state]

    # Tant que il faut encore visiter un état
    while to_visit:
        current_state = to_visit.pop() # Choisir l'un de ceux a visiter
        visited.add(current_state)     # Il a été visité
        for origin in printer.declared_states:          # Pour chaque origine:
            if len(df.loc[df['Origin'] == origin]) == 1: # If there's only one possible action or no actions:
                if df.loc[df['Origin'] == origin, current_state].values[0] > 0: # Si on peut arriver depuis ce état au état analysée
                    if df.loc[df['Origin'] == origin, current_state].values[0] < 1.0: # Si cette probabilitée d'arriver n'est pas sure
                        S_may.add(origin)    # We may arrive
                        if origin in S_never:
                            S_never.remove(origin) # Et on l'envlève des never
                        if origin not in visited:  # Et on doit le visiter
                            to_visit.append(origin)
                    else:                   # Si la probabilité est sure
                        S_sure.add(origin)  # Nous avons un S_sure
                        if origin in S_never:
                            S_never.remove(origin) # Donc pas never
            else: # Si il y a plusieurs actions possibles
                if df.loc[df['Origin'] == origin, current_state].values.sum() > 0: # Et il y a au moins une action avec une probabilité différente de zero d'arriver
                    if 1.0 not in df.loc[df['Origin'] == origin, current_state].values: # Si aucune est de 1
                        S_may.add(origin) # May arrive
                        if origin in S_never:
                            S_never.remove(origin)
                        if origin not in visited:
                            to_visit.append(origin)
                    else:
                        S_sure.add(origin)
                        if origin in S_never:
                            S_never.remove(origin)
    
    return list(S_sure), list(S_may), list(S_never)

In [19]:
smaysurenever = run_mdp(path = "mdp_examples//smaysurenever.mdp", return_printer=True, print_transactions=False) 

ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
ANTLR runtime and generated code versions disagree: 4.11.1!=4.13.1
Initialy declared states: ['S0', 'S1', 'S2', 'S3']
Initialy declared actions: ['a', 'b', 'c']
Transition from S0 with no action and targets ['S1', 'S2'] with weights [1, 1]
Transition from S2 with no action and targets ['S3'] with weights [1]
Transition from S1 with no action and targets ['S3'] with weights [1]

 ---------- WARNINGS WHEN PARSING -----------
( 0 ) - State S0 reward wasn't assigned, using zero as reward
( 1 ) - State S1 reward wasn't assigned, using zero as reward
( 2 ) - State S2 reward wasn't assigned, using zero as reward
( 3 ) - State S3 reward wasn't assigned, using zero as reward


In [21]:
sure, may, never = find_states(smaysurenever, 'S3')

print("Sure: ", sure)
print("May: ", may)
print("Never: ", never)

Sure:  ['S3', 'S2', 'S1']
May:  []
Never:  ['S0']


### Exercice 13:

A partir d'un MDP $ M = (S, \text{Act}, P, \iota_{\text{init}}, AP, L) $ et une propriété $ P_{\text{max}} (\diamond s^*) $ avec $ s^* \in S $ un état donné. Proposer une définition pour la matrice $ A $ et le vecteur $ b $ du programme linéaire $ A \cdot x \geq b $. Calculer $x$.


### Exercice 14

Expliquer comment adapter les algorithmes de vérification de l'exercice 13 pour le calcul de la récompense attendue pour des modèles de récompense Markoviens (MC et MDP).

### Exercice 15
Simulateur de lancers de pièces successifs pour simuler un dé à 6 faces

### Exercice 16
Algorithme de SMC quantitatif - simulant un dé à 6 faces

### Exercice 17.1

Implémenter l’algorithme de SMC qualitatif utilisant SPRT (Sequential Probability Ratio Test [Wald, 1945]) dans le programme de l’exercice 15.

### Exercice 17.2
On appelle $p_{ki}$ la probabilité d’obtenir le chiffre $i$ après $k$ lancers au maximum. Utiliser votre programme pour estimer $P(p_{ki} \geq 0.1)$, $P(p_{ki} \geq 0.14)$, $P(p_{ki} \geq 0.16)$. Qu’observez-vous ? Utiliser $\alpha = \beta = 0.01$.

### Exercice 17.3
Tester sur le modèle du CRAPS.

### Exercice 17.4
Tester sur un MDP

## 3. Modélisation Probabiliste et Apprentissage par Renforcement (Chapitre 3)